In [2]:
import scipy.io
import pandas as pd

mat = scipy.io.loadmat("/disk/home/metabric/BRCA1View20000.mat")

In [3]:
gene_labels = [g[0] for g in mat['gene'][0]]
df = pd.DataFrame(mat['data'].transpose(), columns=gene_labels)
# df['id'] = mat['id'][0]
df['label'] = mat['targets']
df

,SCGB2A2,MUCL1,SCGB1D2,PIP,LOC648852,DNAJA2,TFF3,S100P,CPB1,CEACAM6,...,LOC641311,LOC645307,IL1RAP,LOC647149,LOC642453,LOC652100,LOC646050,LOC644912,LOC652294,label
0,6.365374,5.610561,5.134799,5.735802,5.855315,5.443985,8.005882,5.615179,6.682096,5.756946,...,5.409726,5.420998,5.396492,5.358751,5.424770,5.411055,5.242460,5.197767,5.643537,6
1,14.330442,7.979139,10.244240,7.138279,9.890661,11.195867,13.511164,10.548596,5.491578,6.805606,...,5.213895,5.167652,5.273045,5.287980,5.398061,5.226934,5.396744,5.352158,5.811813,4
2,14.388331,6.016715,12.164435,5.272691,6.468386,5.265518,13.386034,8.764722,7.331663,11.967719,...,5.707537,5.327874,5.549589,5.333051,5.368544,5.296182,5.694022,5.459744,5.269015,4
3,13.111243,7.251862,8.389121,8.664654,10.378585,5.544646,11.159766,8.263224,5.497906,5.923506,...,5.351022,5.009093,5.345468,5.176322,5.326991,5.480259,5.292691,5.242160,5.635111,5
4,12.243103,7.293489,10.919581,9.566137,8.503568,11.492583,11.531352,9.818938,9.443134,6.889670,...,5.320883,5.487783,5.479967,5.332074,5.273141,5.282872,5.222482,5.186136,5.616094,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128,7.199176,5.708891,5.672914,14.464282,9.816897,5.604835,13.303832,6.391817,13.010996,8.294409,...,5.695733,5.363557,5.394837,5.442947,5.423763,5.558225,5.575000,5.636068,5.454823,3
2129,10.354497,14.464282,7.633244,14.464282,11.294018,5.468214,10.886420,12.260033,5.662372,6.274380,...,5.537254,5.613097,5.254552,5.205904,5.486546,5.206480,5.161718,5.451296,5.215587,5
2130,9.294401,13.703049,6.063617,10.063176,5.393440,5.529584,10.937517,12.220400,9.985646,8.370227,...,5.520596,5.572326,5.317225,5.178158,5.316757,5.210332,5.239428,5.395402,5.501265,2
2131,14.464282,13.203708,14.169804,13.731721,12.478907,5.347654,14.464282,9.593148,6.165212,11.207149,...,5.650387,5.359733,5.374522,5.245155,5.453742,5.353135,5.235716,5.155543,5.442698,5


In [4]:
import scipy.io
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# create dataloader
class BaselineMetabricDataset(Dataset):
    def __init__(self, mat_file):
        mat = scipy.io.loadmat(mat_file)
        gene_labels = [g[0] for g in mat['gene'][0]]
        self.df = pd.DataFrame(mat['data'].transpose(), columns=gene_labels)
        # df['id'] = mat['id'][0]
        self.df['label'] = mat['targets']
        self.df['label'] = self.df['label'] - 1
        self.labels = self.df.pop('label')
        self.num_classes = 6
        
        # maybe normalize the columns
        self.tensors = {}
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if idx in self.tensors:
            return self.tensors[idx]
        label = torch.zeros(self.num_classes)
        label[self.labels[idx]] = 1
        sample = (torch.FloatTensor(self.df.iloc[idx, :].values), label)
        self.tensors[idx] = sample
        return sample

In [5]:
mat_file = "/disk/home/metabric/BRCA1View20000.mat"
met_dataset = BaselineMetabricDataset(mat_file)

In [6]:
len(met_dataset)

2133

In [7]:
train_split, test_split = torch.utils.data.random_split(met_dataset, [1900, 233])
train_loader = DataLoader(train_split, batch_size=64, shuffle=True)
test_loader = DataLoader(test_split, batch_size=64, shuffle=True)

In [8]:
print(len(train_loader))
print(len(test_loader))

30
4


In [9]:
IN_DIM = 20000
OUT_DIM = 6
LEARNING_RATE = 1e-5
N_EPOCHS = 100
BATCH_SIZE = 100

model = nn.Sequential(
    nn.Linear(IN_DIM, 500),
    nn.ReLU(),
    nn.Linear(500, 100),
    nn.ReLU(),
    nn.Linear(100, OUT_DIM)
)

loss_fn = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(N_EPOCHS):
    loss = None
    for i, (X, y) in enumerate(train_loader):
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
    print(epoch, loss.item())

0 33.4311408996582
1 33.842437744140625
2 29.600605010986328
3 33.3281364440918
4 33.29365921020508
5 28.77070426940918
6 30.155563354492188
7 27.86324691772461
8 30.065120697021484
9 27.8597354888916
10 27.394798278808594
11 27.335826873779297
12 27.84022331237793
13 24.21585464477539
14 26.190759658813477
15 24.8209171295166
16 27.13934326171875
17 22.396656036376953
18 25.534509658813477
19 25.666337966918945
20 23.255090713500977
21 25.254640579223633
22 25.548118591308594
23 21.12644386291504
24 22.975475311279297
25 20.779685974121094
26 20.59823989868164
27 20.567575454711914
28 22.910226821899414
29 19.635847091674805
30 24.309661865234375
31 25.80802345275879
32 19.79580307006836
33 21.617877960205078
34 19.828155517578125
35 19.519474029541016
36 23.325336456298828
37 19.340192794799805
38 20.15607261657715
39 22.60725975036621
40 21.07512092590332
41 21.386354446411133
42 22.29369354248047
43 24.881759643554688
44 19.816030502319336
45 18.97570037841797
46 22.79999542236328


In [11]:
correct = 0
total = 0
with torch.no_grad():
    for X, y in test_loader:
        predictions = model(X)
        _, pred = torch.max(predictions.data, 1)
        _, labeled = torch.max(y.data, 1)
        total += y.size(0)
        correct += (pred == labeled).sum().item()
        print(pred)
        print(labeled)
        print("---------------------------------")
print(f"Accuracy: {correct} / {total} {100 * correct / total:.2f}%")


tensor([1, 2, 0, 1, 0, 5, 2, 3, 3, 2, 4, 3, 2, 0, 2, 2, 1, 2, 3, 2, 0, 0, 3, 2,
        2, 0, 1, 3, 0, 2, 3, 1, 2, 3, 2, 2, 2, 3, 4, 1, 1, 1, 3, 1, 0, 2, 3, 2,
        2, 4, 2, 5, 0, 3, 5, 1, 3, 0, 0, 2, 3, 2, 0, 4])
tensor([1, 4, 0, 1, 0, 5, 2, 3, 3, 2, 4, 3, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 3, 2,
        3, 0, 1, 3, 0, 2, 1, 0, 3, 3, 3, 3, 2, 3, 4, 1, 1, 0, 3, 1, 0, 3, 3, 2,
        2, 1, 3, 5, 0, 3, 5, 0, 3, 0, 0, 2, 3, 4, 0, 4])
---------------------------------
tensor([0, 2, 2, 2, 2, 3, 3, 2, 3, 3, 2, 3, 3, 3, 1, 3, 1, 2, 2, 2, 2, 1, 3, 3,
        3, 2, 2, 0, 2, 3, 3, 1, 0, 3, 0, 0, 2, 1, 2, 2, 2, 2, 5, 3, 2, 2, 1, 0,
        2, 3, 3, 1, 0, 3, 5, 2, 3, 2, 2, 2, 1, 1, 0, 0])
tensor([0, 2, 2, 2, 2, 3, 3, 2, 1, 3, 1, 3, 3, 1, 1, 2, 0, 2, 0, 2, 2, 1, 3, 3,
        3, 4, 4, 0, 2, 2, 3, 1, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1, 5, 1, 3, 2, 0, 0,
        2, 3, 3, 1, 0, 3, 5, 2, 3, 2, 2, 4, 1, 0, 0, 0])
---------------------------------
tensor([2, 2, 4, 3, 5, 3, 3, 3, 2, 2, 3, 2, 4, 2, 1, 2, 4, 2, 2,

In [36]:
torch.save(model.state_dict(), 'baseline_model_v0.2.pth')